In [1]:
# Get the data here: https://data.world/dataquest/mlb-game-logs
import sys, os
sys.path.insert(0, os.path.abspath("../.."))
%load_ext ElasticNotebook

ModuleNotFoundError: No module named 'xxhash'

In [ ]:
%%RecordEvent
import pandas as pd
gl = pd.read_csv('data/game_logs.csv')
gl.head()

In [6]:
%%RecordEvent
gl = gl.fillna(0)
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 859.4 MB
gl
bytestring b'\x80\x04\x95\x01\x00\x01\x00\x00\x00\x00\x00\x8c\x1eelastic.core.mutation.id_graph\x94\x8c\x0bIdGraphNode\x94\x93\x94)\x81\x94}\x94(\x8c\x06obj_id\x94\x8a\x06\x10\xdf\xa9\x9a\xf0\x7f\x8c\x08obj_type\x94\x8c\tDataFrame\x94\x8c\x0bchild_nodes\x94]\x94(Nh\x02)\x81\x94}\x94(h\x05\x8a\x06\x80\x9b9\xb8\xf0\x7fh\x06\x8c\x0cBlockManager\x94h\x08]\x94ubh\x02)\x81\x94}\x94(h\x05\x8a\x06@\xd6\xac\xc8\xf0\x7fh\x06\x8c\x04dict\x94h\x08]\x94(h\x02)\x81\x94}\x94(h\x05\x8a\x06p\xcc!\xe8\xf0\x7fh\x06\x8c\x03str\x94h\x08]\x94ubh\x02)\x81\x94}\x94(h\x05\x8a\x06\xc0A\x83\xea\xf0\x7fh\x06\x8c\x06Series\x94h\x08]\x94(Nh\x02)\x81\x94}\x94(h\x05\x8a\x06po\x97\xf9\xf0\x7fh\x06\x8c\x12SingleBlockManager\x94h\x08]\x94(h\x02)\x81\x94}\x94(h\x05\x8a\x06\x80\xacv\xb8\xf0\x7fh\x06\x8c\x04list\x94h\x08]\x94h\x02)\x

In [7]:
%%RecordEvent
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,0,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,0,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,0,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


gl
pd
modified variables: set()


In [4]:
%%RecordEvent
for dtype in ['float','int','object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 1.29 MB
Average memory usage for int columns: 1.12 MB
Average memory usage for object columns: 9.50 MB


In [5]:
%%RecordEvent
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [6]:
%%RecordEvent
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(gl_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

7.87 MB
1.48 MB


,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [7]:
%%RecordEvent
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(gl_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([gl_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

100.99 MB
50.49 MB


,before,after
float32,NaN,77.0
float64,77.0,NaN


In [8]:
%%RecordEvent
optimized_gl = gl.copy()
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

print(mem_usage(gl))
print(mem_usage(optimized_gl))

859.43 MB
802.54 MB


In [9]:
%%RecordEvent
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,forefeit,protest,park_id,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,171907,171907,171907,171907,171907,140150,116,145,180,171907,...,140838,140838,140838,140838,140838,140838,140838,140838,1456,140841
unique,7,148,7,148,7,2,116,3,5,245,...,4774,4720,5253,5197,4760,4710,5193,5142,332,1
top,Sat,CHN,NL,CHN,NL,D,"19200904,,0,6,36",H,V,STL07,...,grimc101,Charlie Grimm,grimc101,Charlie Grimm,lopea102,Al Lopez,spahw101,Warren Spahn,HTBF,Y
freq,28891,8870,88866,9024,88867,82724,1,69,90,7022,...,427,427,491,491,676,676,339,339,1112,140841


In [10]:
%%RecordEvent
dow = gl_obj.day_of_week
print(dow.head())
dow_cat = dow.astype('category')
print(dow_cat.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object
0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): ['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']


In [11]:
%%RecordEvent
converted_obj = pd.DataFrame()
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [13]:
%Checkpoint checkpoints/pandas.pickle


---------------------------
variables to migrate:
---------------------------
variables to recompute:
gl 901174675
compare_ints 1836
converted_float 52947500
mean_usage_b 32
col 65
dow 10314564
converted_obj 52313108
mem_usage 136
compare_floats 23632
mean_usage_mb 32
converted_int 1547307
optimized_gl 841522946
gl_float 105894856
num_total_values 28
dtype 55
selected_dtype 787028427
pd 72
dow_cat 172771
gl_int 8251680
num_unique_values 28
gl_obj 787028427
---------------------------
OEs to recompute:
4 0.02417898178100586
9 1.182535171508789
0 2.0505001544952393
5 0.036476850509643555
3 0.0004811286926269531
6 1.8967838287353516
7 0.8152618408203125
8 0.006530046463012695
2 0.6810100078582764
Checkpoint saved to: checkpoints/pandas.pickle
